## Directories etc

In [5]:
from __future__ import division

import codecs
import csv
import os
import time
from os import listdir
from os.path import isfile, join
import re
import io


# Directory and file names:

# ENGLISH
#dir_in = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..", "hartlib","OneDrive", "lemmatization", "input_TM"))
#dir_out = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..", "hartlib","OneDrive", "files_years", "en"))  # relative path to data directory
#dir_hartlib = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..","hartlib", "OneDrive", "Hartlib Papers 2nd edition"))


# LATIN
dir_in = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..", "hartlib","OneDrive", "lemmatization", "la","output"))
dir_out = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..", "hartlib","OneDrive", "files_years", "la"))  # relative path to data directory
dir_hartlib = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..","hartlib", "OneDrive", "Hartlib Papers 2nd edition"))



# create output directory if it doesn't exist:
if not os.path.exists(dir_out):
    os.makedirs(dir_out)

    
num_topics = 40

In [6]:
print(dir_in)
print(dir_out)
print(dir_hartlib)
#/Users/hengchen/hartlib/OneDrive

/Users/hengchen/hartlib/OneDrive/lemmatization/la/output
/Users/hengchen/hartlib/OneDrive/files_years/la
/Users/hengchen/hartlib/OneDrive/Hartlib Papers 2nd edition


## Get date from the files in Hartlib 2nd edition

In [7]:
file_year = {}
missing = list()
files_hartlib = [f for f in listdir(dir_hartlib) if isfile(join(dir_hartlib, f))]
for file in files_hartlib:   # for each file in the folder, we open hit and retrieve a date that is possible for us ([1][6][0-9]{2}).
    print("file:" + file)
    text = open(os.path.join(dir_hartlib, file), 'r').readlines()
    i = 0
    for line in text:
        i += 1
        if i == 1:
            hit = re.search("[1][6][0-9]{2}",line) # We can't utilise the HTML because tags differ. Also, sometimes no true date but a guess (a full text)
        
            if hit:
                if hit.group(0) != "1000":
                    file_year[file[:-5]] = hit.group(0) # we remove ".html" and use that as dictionary key
            else :
                missing.append(file)

set_missing = set(missing)    

print("missing a date:")
print(len(set_missing))
#print(missing)        
print("letters with a date:")
print(len(file_year))
print("letters in the hartlib folder:")
print(len(files_hartlib))
#print(file_year)

file:10A_01_01.html
file:10A_01_03.html
file:10A_01_05.html
file:10A_01_06.html
file:10A_01_08.html
file:10A_02_01.html
file:10A_02_02.html
file:10A_02_04.html
file:10A_02_06.html
file:10A_02_07.html
file:10A_02_09.html
file:10A_02_10.html
file:10A_02_12.html
file:10A_02_16.html
file:10A_02_18.html
file:10A_02_20.html
file:10A_02_21.html
file:10A_02_23.html
file:10A_02_25.html
file:10A_02_27.html
file:10A_02_28.html
file:10A_02_30.html
file:10A_02_31.html
file:10A_02_32.html
file:10A_02_33.html
file:10A_02_34.html
file:10A_02_35.html
file:10A_02_37.html
file:10A_02_40.html
file:10A_02_43.html
file:10A_02_45.html
file:10A_02_47.html
file:10B_03.html
file:10B_04.html
file:10B_05_01.html
file:10B_05_03.html
file:10B_05_04.html
file:10B_05_06.html
file:10B_05_07.html
file:10B_05_09.html
file:10B_05_11.html
file:10B_05_13.html
file:10B_05_14.html
file:10B_05_16.html
file:10B_05_19.html
file:10B_05_21.html
file:10B_05_23.html
file:10B_05_25.html
file:10B_05_27.html
file:10B_06.html
file:10B_

file:16B_02_30.html
file:16B_02_31.html
file:16B_02_32.html
file:16B_02_33.html
file:16B_03.html
file:16B_04.html
file:16B_05.html
file:16B_06_01.html
file:16B_06_02.html
file:16B_06_03.html
file:16B_06_04.html
file:16B_06_05.html
file:16B_06_06.html
file:16B_07.html
file:16B_08.html
file:16B_09_01.html
file:16B_09_03.html
file:16B_09_09.html
file:16B_09_11.html
file:16B_09_17.html
file:16B_09_19.html
file:16B_09_21.html
file:16B_09_23.html
file:16B_09_25.html
file:16B_10_01.html
file:16B_10_02.html
file:17_01_01.html
file:17_01_02.html
file:17_02.html
file:17_03.html
file:17_04.html
file:17_05.html
file:17_06.html
file:17_07_01.html
file:17_07_05.html
file:17_08.html
file:17_09.html
file:17_10.html
file:17_11.html
file:17_12.html
file:17_13.html
file:17_14_01.html
file:17_14_02.html
file:17_15.html
file:17_16.html
file:17_17.html
file:17_18.html
file:17_19_01.html
file:17_19_02.html
file:18A_01_01.html
file:18A_01_02.html
file:18A_01_04.html
file:18A_01_05.html
file:18A_01_07.html
fil

file:24_08_01.html
file:24_08_11.html
file:24_09_01.html
file:24_09_09.html
file:24_10_01.html
file:24_10_10.html
file:24_11.html
file:24_12.html
file:24_13_01.html
file:24_13_10.html
file:24_13_16.html
file:24_13_23.html
file:24_13_30.html
file:24_14.html
file:24_15_01.html
file:24_15_14.html
file:24_16_01.html
file:24_16_07.html
file:24_16_15.html
file:24_16_22.html
file:24_16_32.html
file:24_16_42.html
file:24_16_52.html
file:24_17.html
file:24_18_01.html
file:24_18_13.html
file:24_19.html
file:25_01.html
file:25_02_01.html
file:25_02_02.html
file:25_03_01.html
file:25_03_02.html
file:25_03_04.html
file:25_03_05.html
file:25_03_07.html
file:25_04.html
file:25_05_01.html
file:25_05_13.html
file:25_05_21.html
file:25_05_25.html
file:25_06.html
file:25_07_01.html
file:25_07_05.html
file:25_08_01.html
file:25_08_03.html
file:25_09.html
file:25_10.html
file:25_11.html
file:25_12_01.html
file:25_12_08.html
file:25_12_16.html
file:25_12_25.html
file:25_12_35.html
file:25_13.html
file:25_14

file:31D_21_09.html
file:31D_21_10.html
file:31D_22_01.html
file:31D_22_14.html
file:31D_22_27.html
file:31D_23_01.html
file:31D_23_02.html
file:31D_23_04.html
file:31D_23_05.html
file:31D_23_06.html
file:31D_23_10.html
file:31D_23_11.html
file:31D_23_13.html
file:31D_23_15.html
file:31D_23_17.html
file:31D_23_19.html
file:31D_23_21.html
file:31D_23_23.html
file:31D_23_25.html
file:31D_23_26.html
file:31D_23_28.html
file:31D_23_32.html
file:31D_23_36.html
file:31D_23_38.html
file:31D_23_40.html
file:31D_23_41.html
file:31D_23_43.html
file:31D_23_47.html
file:31D_23_49.html
file:31D_23_52.html
file:31D_23_56.html
file:32A_01_01.html
file:32A_01_06.html
file:32A_01_08.html
file:32A_01_10.html
file:32A_01_12.html
file:32A_01_13.html
file:32A_02_01.html
file:32A_02_05.html
file:32A_02_07.html
file:32A_02_09.html
file:32A_02_13.html
file:32A_02_15.html
file:32A_02_17.html
file:32A_02_19.html
file:32A_02_21.html
file:32A_02_23.html
file:32A_02_25.html
file:32A_02_26.html
file:32A_02_28.html


file:3A_02_025.html
file:3A_02_027.html
file:3A_02_028.html
file:3A_02_031.html
file:3A_02_032.html
file:3A_02_033.html
file:3A_02_034.html
file:3A_02_036.html
file:3A_02_038.html
file:3A_02_039.html
file:3A_02_041.html
file:3A_02_042.html
file:3A_02_043.html
file:3A_02_045.html
file:3A_02_047.html
file:3A_02_049.html
file:3B_02_051.html
file:3B_02_052.html
file:3B_02_054.html
file:3B_02_055.html
file:3B_02_057.html
file:3B_02_058.html
file:3B_02_059.html
file:3B_02_060.html
file:3B_02_064.html
file:3B_02_065.html
file:3B_02_066.html
file:3B_02_067.html
file:3B_02_069.html
file:3B_02_070.html
file:3B_02_074.html
file:3B_02_076.html
file:3B_02_077.html
file:3B_02_079.html
file:3B_02_081.html
file:3B_02_082.html
file:3B_02_083.html
file:3B_02_084.html
file:3B_02_085.html
file:3B_02_086.html
file:3B_02_088.html
file:3B_02_090.html
file:3B_02_091.html
file:3B_02_092.html
file:3B_02_096.html
file:3B_02_098.html
file:3B_02_100.html
file:3B_02_101.html
file:3B_02_103.html
file:3B_02_105.html


file:49A_13_05.html
file:49A_14.html
file:49A_15.html
file:49A_16_01.html
file:49A_16_03.html
file:49A_16_05.html
file:49A_16_07.html
file:49A_16_09.html
file:49A_16_10.html
file:49A_17.html
file:49A_18_01.html
file:49A_18_03.html
file:49A_18_05.html
file:49A_18_07.html
file:49A_19_01.html
file:49A_19_02.html
file:49A_20.html
file:49A_21.html
file:49A_22_01.html
file:49A_22_03.html
file:49A_22_04.html
file:49A_22_06.html
file:49A_22_08.html
file:49B_23.html
file:49B_24_01.html
file:49B_24_02.html
file:49B_25.html
file:49B_26.html
file:49B_27.html
file:49B_28.html
file:49B_29_01.html
file:49B_29_03.html
file:49B_30.html
file:49B_31_01.html
file:49B_31_03.html
file:49B_32.html
file:49B_33.html
file:49B_34.html
file:49B_35_01.html
file:49B_35_03.html
file:49B_36.html
file:49B_37_01.html
file:49B_37_02.html
file:49B_38_01.html
file:49B_38_02.html
file:49B_39.html
file:49B_40.html
file:49B_41_01.html
file:49B_41_02.html
file:49B_41_03.html
file:49B_41_05.html
file:49B_42.html
file:49B_43.ht

file:5A_04_01.html
file:5A_04_07.html
file:5A_05_01.html
file:5A_05_03.html
file:5A_05_05.html
file:5A_05_07.html
file:5A_05_09.html
file:5A_05_10.html
file:5A_05_12.html
file:5A_06_01.html
file:5A_06_02.html
file:5A_07.html
file:5A_08.html
file:5A_09.html
file:5B_10.html
file:5B_11.html
file:5B_12_01.html
file:5B_12_03.html
file:5B_12_05.html
file:5B_12_09.html
file:5B_12_13.html
file:5B_12_17.html
file:5B_12_19.html
file:5B_12_21.html
file:5B_12_23.html
file:5B_13.html
file:5B_14.html
file:5B_15.html
file:5B_16_01.html
file:5B_16_03.html
file:5B_17.html
file:5B_18.html
file:5B_19.html
file:5B_20.html
file:5B_21.html
file:5B_22.html
file:5B_23.html
file:5B_24.html
file:5B_25.html
file:5B_26_01.html
file:5B_26_03.html
file:5B_26_04.html
file:5B_26_05.html
file:5B_26_07.html
file:5B_26_09.html
file:5B_26_11.html
file:5B_27_01.html
file:5B_27_03.html
file:5B_27_05.html
file:5C_28_01.html
file:5C_28_03.html
file:5C_28_11.html
file:5C_28_13.html
file:5C_29.html
file:5C_30.html
file:5C_31.h

file:8A_07_02.html
file:8A_07_03.html
file:8A_08.html
file:8A_09.html
file:8A_10.html
file:8A_11.html
file:8A_12.html
file:8A_13.html
file:8A_14.html
file:8A_15.html
file:8A_16.html
file:8A_17.html
file:8A_18.html
file:8A_19.html
file:8A_20.html
file:8A_21.html
file:8A_22.html
file:8A_23.html
file:8A_24_01.html
file:8A_24_03.html
file:8A_25.html
file:8A_26_01.html
file:8A_26_02.html
file:8A_27.html
file:8A_28.html
file:8A_29.html
file:8B_30_01.html
file:8B_30_02.html
file:8B_31_01.html
file:8B_31_04.html
file:8B_31_09.html
file:8B_32.html
file:8B_33.html
file:8B_34.html
file:8B_35.html
file:8B_36.html
file:8B_37.html
file:8B_38.html
file:8B_39.html
file:8B_40_01.html
file:8B_40_03.html
file:8B_40_05.html
file:8B_40_06.html
file:8B_40_08.html
file:8B_40_11.html
file:8B_40_13.html
file:8B_40_14.html
file:8B_41.html
file:8B_42.html
file:8B_43.html
file:8B_44.html
file:8B_45.html
file:8B_46.html
file:8B_47.html
file:8B_48.html
file:8B_49.html
file:8B_50.html
file:8B_51.html
file:8B_52.html

## Getting the files in {language} to match them with the letters with a date
{language} being defined in the first cell

In [8]:
# We first recreate a list with the file ID, not the filename

files_in = [f for f in listdir(dir_in) if isfile(join(dir_in, f))]
print(len(files_in))
files_in_no = list()

for file in files_in:  # this creates a list with the file ID, not the filename
    #print(file[:-12])
    #files_in_no.append(file[:-12])      ### THIS IS FOR ENGLISH
    files_in_no.append(file[:-22])       ### THIS IS FOR LATIN 
    
    
    
#print(len(files_in_no))

# We then match

dico_en = dict()
liste_en_no_date = list()



#print(files_in_no)
#print("\n\n\n")
#print(missing)
z = 0
for chose in files_in_no:
    if chose in file_year:
        dico_en[chose] = file_year[chose]
        z = z+1


print(z)

for fichier in files_in_no:
    fichier = fichier+".html"
    if fichier in missing:
        liste_en_no_date.append(fichier)

print("{language} files with date:")
print(len(dico_en))
#print(dico_en)
print("{language} files with no date:")
print(len(liste_en_no_date))
#print(liste_en_no_date)

# here we check if the letters that don't have a date contain "reply", as sometimes the replies do contain a date

#print("dico EN: ",dico_en)

#x = 0

#for file in files_hartlib:
#    if file in liste_en_no_date:
#        fichier = open(dir_hartlib+"/"+file)
#        texte = fichier.readlines()
#        for line in texte:
#            if "reply to" in line:
#                print("oui "+file)
#                x = x+1
#print(x)



1536
946
{language} files with date:
946
{language} files with no date:
590


# Using the results of LDA
## We create dict_topic_year[filename] = [topic, year]  (might be useless as only one entry per file, recheck)

In [9]:
# this might be wrong as "several topics per file" !

#topics_csv = open(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "OneDrive", "TM_output", "40", "output_csv","DocsInTopics.csv")),"r")

### Below is for English
#topics_csv = open(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..","hartlib", "OneDrive", "TM_output_2", "40", "output_csv","DocsInTopics.csv")),"r")

### Below is for Latin
topics_csv = open(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..","hartlib", "OneDrive", "TM_output", "TM_output_LA3", "40", "output_csv","DocsInTopics.csv")),"r")

topics_csv = topics_csv.readlines()
print("Number of lines in the CSV: "+str(len(topics_csv)))

dict_topic_year = dict()

for line in topics_csv:
    line = line.rstrip()
    line = line.split(";")
    file_name = line[3]    # we retrieve the file name
    
    ###  This below is for English
    #file_name = file_name[64:-12]  # we keep the ID and remove the path
    
    ### This below is for Latin
    file_name = file_name[53:-22]
    
    if file_name in file_year:
        dict_topic_year[file_name] = [line[0],file_year[file_name]]  # we update the dictionary with filename as key, topic and year as data

print("Number of entries in the dictionary :"+str(len(dict_topic_year)))        
print(dict_topic_year)

Number of lines in the CSV: 20001
Number of entries in the dictionary :869
{'1B_33_039': ['39', '1648'], '1C_33_079': ['39', '1644'], '1B_33_022': ['39', '1648'], '1C_33_044': ['39', '1648'], '1C_33_094': ['39', '1648'], '1B_33_007': ['39', '1647'], '1C_33_099': ['39', '1648'], '7D_078': ['37', '1640'], '1C_33_067': ['39', '1648'], '1B_33_012': ['39', '1647'], '55A_01': ['39', '1634'], '26B_24': ['39', '1638'], '19A_01_20': ['39', '1622'], '1C_33_052': ['39', '1648'], '1B_33_033': ['37', '1648'], '1C_35_05': ['37', '1647'], '1C_36_07': ['36', '1648'], '19A_01_33': ['39', '1633'], '1C_33_102': ['39', '1649'], '12A_017': ['38', '1646'], '1C_34_11': ['39', '1647'], '56A_01_059': ['39', '1657'], '20A_01_28': ['39', '1635'], '7C_046': ['36', '1641'], '5C_29': ['38', '1639'], '18C_20': ['37', '1637'], '1B_33_026': ['39', '1648'], '20B_10_02': ['39', '1654'], '45B_03_23': ['39', '1642'], '26A_06': ['39', '1657'], '1B_33_003': ['34', '1646'], '18C_23': ['39', '1638'], '4A_01_01': ['37', '1647'

## Using topicsindoc.csv, take all top topics and the files that pertain to them
### Also, given the time slices, we create topic_slice_files which maps (topic, slice) to the corresponding files

In [10]:
#topicsindocs_csv = open(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "OneDrive", "TM_output", "40", "output_csv","TopicsInDocs.csv")),"r")
topicsindocs_csv = open(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..","hartlib", "OneDrive", "TM_output", "TM_output_LA3", "40", "output_csv","TopicsInDocs.csv")),"r")
topicsindocs_csv = topicsindocs_csv.readlines()[1:] # skip the first line (headers)
dict_file_topic = dict()
list_topics = list()
topic_slice_files = dict() # maps a (topic, time slice) to the corresponding list of files

# slices = [1610,1635,1650,1675]
slices = [1610,1650,1665] # now trying with two slices

topic_year_number = dict()

## dico_en is file_year for ENGLISH

liste_annee = list(dico_en.values())

liste_annee_liste = liste_annee
liste_annee = list(set(liste_annee)) # all different years for which we have a file
#print(liste_annee)

print("\n\nTime Range: "+str(sorted(liste_annee)[0])+" --> "+str(sorted(liste_annee)[len(liste_annee)-1]))
total_of_years = int(sorted(liste_annee)[len(liste_annee)-1])-int(sorted(liste_annee)[0])
print("total of years",total_of_years,"\n")

yearz = list()
for i in range(int(sorted(liste_annee)[0]),int(sorted(liste_annee)[len(liste_annee)-1])):  # this is just to get all dates
    yearz.append(i)
print(yearz)

annee_count = dict()
for annee in liste_annee:
    annee_count[annee] = 0

for i in range(0,40):
    list_topics.append(i)
    dict_file_topic[i] = 0
#    topic_year_number[i] = dict(annee_count)  # for each of the topic number we have a dictionary with the year as key
#    for annee in liste_annee: 
        


for line in topicsindocs_csv:  # we discriminate on FILES
    line = line.rstrip()
    line = line.split(";")
   # print(line)
    
#    print(line) isinstance(1,int)
        
        #print(topic_number)
    #    x = 0
    
    #print(line)
    file_name_indocs = line[1]    # we retrieve the file name
    
    ### This is for English
    #file_name_indocs = file_name_indocs[64:-12]  # we keep the ID and remove the path
    
    ### This is for Latin
    file_name_indocs = file_name_indocs[53:-22]  # we keep the ID and remove the path
    
    
    j = 0
    topics_this_line = list()
    for col in line: 
        #print(col)
            
        if j % 2 == 0 and j > 0:  # we need j is at least 2, and even to capture the columns with topics
            topic = col

            if topic != "":
                topics_this_line.append(topic)
            
        j += 1
    
    for topic_number in topics_this_line:
        #print(topic_number)
        if file_name_indocs in file_year:
            year = file_year[file_name_indocs]
           
        
        
            if (topic_number, year) in topic_year_number:  # this is for the count
                topic_year_number[(topic_number, year)] += 1
            
            if (topic_number, year) not in topic_year_number:  # this is for the count
                topic_year_number[(topic_number, year)] = 1
                
            
            for s in range(0,len(slices)-1):
                if int(year) > slices[s] and int(year) <= slices[s+1]:
                    if (str(topic_number),s) in topic_slice_files.keys():
                        topic_slice_files[(str(topic_number),s)].append(file_name_indocs)
                    else:                        
                        topic_slice_files[(str(topic_number),s)] = [file_name_indocs]
                        print("topic number ",str(topic_number),"slice ",str(s),file_name_indocs)
                
                            
                    
    
#print(topic_year_number)
#for (topic, year) in topic_year_number: 
    
#    une_annee = topic_year_number[topic, year] 
#    if une_annee >= 50:
#        print("topic",topic,"year",year,"nombre une_annee",une_annee)
        # this above prints the topics for which, at a given year, there are at least 50 files
        
       
    # topic_year_number{} (topic,year): numberoffiles

    
#print("number of entries in topic_year_number:",len(topic_year_number.keys()))
# average of 28 years per topic


#for t in range(0,40):  # for each topic 
#    print("\n\ntopic",t)
#    comptage = 0
#
    
#    for key in topic_year_number.keys():  # for every key
#        if int(key[0]) == t:  # if int(first item in key) is the t we're currently dealing with
    
#            for y in range(0,total_of_years):  # for all values y between 0 and the total numbers of years we have
#                comptage = 0
                       
#                cette_annee = (key[0],key[1])[1]  # storing, in cette_annee, the year for topic t  (iteration)
                


                #if (key[0],cette_annee) in topic_year_number:  # if the key exists
                 #   print(key[0],cette_annee,topic_year_number[(key[0],cette_annee)])
            
                # if this date matches one of the dates we have AND the entry exists
#                if int(cette_annee) == yearz[y] and (key[0],cette_annee) in topic_year_number:  
                  
#                    comptage = comptage + topic_year_number[(key[0],cette_annee)] # we count
#                    print(cette_annee,comptage)
               
                




Time Range: 1610 --> 1669
total of years 59 

[1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1665, 1666, 1667, 1668]
topic number  13 slice  0 10B_05_01
topic number  18 slice  0 10B_05_01
topic number  12 slice  0 10B_05_01
topic number  23 slice  0 10B_05_01
topic number  20 slice  0 11A_01_001
topic number  39 slice  0 11A_01_003
topic number  36 slice  0 11A_01_003
topic number  22 slice  0 11A_01_008
topic number  16 slice  0 11A_01_008
topic number  35 slice  0 11A_01_008
topic number  21 slice  0 11A_01_011
topic number  6 slice  0 11A_01_013
topic number  1 slice  0 11A_01_013
topic number  3 slice  0 11A_01_014
topic number  8 slice  0 11A_01_014
topic number  7 slice  0 11A_01_020
topic number  

In [11]:
print(len(topic_slice_files))
print(sorted(topic_slice_files[('0', 0)]))
#print(len(topic_slice_files[('0', 0)]))
#print(topic_slice_number['0', 0])

80
['11A_01_023', '11A_01_042', '11B_01_068', '11B_01_093', '11C_01_116', '11C_01_119', '12A_017', '12A_018', '12A_022', '12A_028', '18B_02_40', '18B_03_12D', '18C_20', '18C_23', '19A_01_33', '1A_02_05', '1A_22', '1B_33_003', '1B_33_006', '1B_33_007', '1B_33_012', '1B_33_018', '1B_33_020', '1B_33_022', '1B_33_026', '1B_33_033', '1B_33_039', '1C_33_043', '1C_33_044', '1C_33_052', '1C_33_056', '1C_33_060', '1C_33_062', '1C_33_067', '1C_33_068', '1C_33_074', '1C_33_094', '1C_33_099', '1C_33_102', '1C_34_11', '1C_34_13', '1C_35_05', '1C_36_07', '20A_01_28', '20A_04_15', '23A_05', '26B_24', '31B_12_01', '44A_01_08', '44A_01_11', '45B_03_19', '45B_03_20', '45B_03_23', '45B_03_25', '45C_04_01', '45C_04_03', '45C_04_04', '46A_06_03', '47A_09_34', '49A_02_01', '49A_03', '49A_08', '49A_10', '49B_24_02', '49B_30', '49B_32', '49B_33', '49B_45_01', '49B_45_03', '4A_01_01', '58_02', '59B_10_078', '5A_04_07', '5B_12_21', '5D_45_04', '5D_47', '5D_49', '5D_53_11', '5D_53_12', '6C_09', '7A_014', '7A_015

## Defining time slices
Getting for each topic the number of files per time slice

In [12]:
print(topic_year_number)

{('13', '1650'): 7, ('18', '1650'): 1, ('12', '1650'): 5, ('23', '1650'): 1, ('13', '1635'): 25, ('20', '1635'): 14, ('39', '1634'): 8, ('36', '1634'): 2, ('13', '1634'): 18, ('12', '1634'): 7, ('20', '1634'): 5, ('22', '1634'): 6, ('16', '1634'): 2, ('35', '1634'): 5, ('21', '1634'): 4, ('6', '1634'): 2, ('1', '1634'): 5, ('3', '1634'): 4, ('8', '1634'): 1, ('7', '1634'): 2, ('0', '1634'): 4, ('14', '1634'): 4, ('3', '1635'): 18, ('4', '1635'): 11, ('17', '1635'): 15, ('1', '1635'): 15, ('37', '1635'): 2, ('30', '1635'): 3, ('16', '1635'): 2, ('15', '1635'): 3, ('39', '1635'): 11, ('31', '1635'): 5, ('14', '1635'): 5, ('34', '1634'): 3, ('18', '1634'): 4, ('17', '1634'): 9, ('9', '1635'): 9, ('0', '1635'): 5, ('22', '1635'): 4, ('29', '1635'): 1, ('36', '1635'): 2, ('10', '1635'): 1, ('23', '1635'): 3, ('3', '1636'): 1, ('13', '1636'): 4, ('4', '1636'): 9, ('1', '1636'): 3, ('17', '1636'): 3, ('37', '1636'): 9, ('35', '1636'): 3, ('20', '1636'): 2, ('23', '1636'): 4, ('34', '1636'): 1

topic_slice_number = dict() #maps a (topic, time slice) to the corresponding number of files



for topic in range(40):  # for each topic
    topic = str(topic)
    print("\ntopic:",topic)
    for i in range(0,len(slices)-1): # for each of the slices
        print("slice number:", i)
    

        topic_slice_number[(topic, i)] = 0

        for x in range(slices[i],slices[i+1]):
            #print(x)
            x = str(x)
            if (topic,x) in topic_year_number:
                #print(topic_year_number[(topic,x)])
                topic_slice_number[(topic, i)] += topic_year_number[(topic,x)]
                                              
                # todo here append file names to topic_slice_files
                
        
        if topic_slice_number[(topic, i)] != 0:
            print(topic_slice_number[(topic, i)])
#topic_slice_number[('0',1)]            

In [13]:
topic_abc = dict()
for (t,s) in topic_slice_files:
    topic_abc[(t,s)] = len(topic_slice_files[(t,s)])

print(topic_abc[('0',0)])

100


## Copy the files to their corresponding folders slice/topic 


In [14]:
#print(destination)
import shutil
from shutil import copyfile
topic_slice_files

{('0', 0): ['11A_01_023',
  '11A_01_042',
  '11B_01_068',
  '11B_01_093',
  '11C_01_116',
  '11C_01_119',
  '12A_017',
  '12A_018',
  '12A_022',
  '12A_028',
  '18B_02_40',
  '18B_03_12D',
  '18C_20',
  '18C_23',
  '19A_01_33',
  '1A_02_05',
  '1A_22',
  '1B_33_003',
  '1B_33_006',
  '1B_33_007',
  '1B_33_012',
  '1B_33_018',
  '1B_33_020',
  '1B_33_022',
  '1B_33_026',
  '1B_33_033',
  '1B_33_039',
  '1C_33_043',
  '1C_33_044',
  '1C_33_052',
  '1C_33_056',
  '1C_33_060',
  '1C_33_062',
  '1C_33_067',
  '1C_33_068',
  '1C_33_074',
  '1C_33_094',
  '1C_33_099',
  '1C_33_102',
  '1C_34_11',
  '1C_34_13',
  '1C_35_05',
  '1C_36_07',
  '20A_01_28',
  '20A_04_15',
  '23A_05',
  '26B_24',
  '31B_12_01',
  '44A_01_08',
  '44A_01_11',
  '45B_03_19',
  '45B_03_20',
  '45B_03_23',
  '45B_03_25',
  '45C_04_01',
  '45C_04_03',
  '45C_04_04',
  '46A_06_03',
  '47A_09_34',
  '49A_02_01',
  '49A_03',
  '49A_08',
  '49A_10',
  '49B_24_02',
  '49B_30',
  '49B_32',
  '49B_33',
  '49B_45_01',
  '49B_45_

In [15]:
number_files_slices = list()
total_number_files = 0

for i in range(0,len(slices)-1):
    number_files_slices.append(0)

#)

for topic in range(0,num_topics):
    topic = str(topic)
    for i in range(0,len(slices)):
        if (topic,i) in topic_slice_files:
            for filename in topic_slice_files[(topic,i)]:
                #print(filename)
                
                ### THIS BELOW FOR ENGLISH
               # source = dir_in+"/"+filename+"_cleaned.txt"
                ### THIS BELOW FOR LATIN
                source = dir_in+"/"+filename+"_cleaned_la_lemmas.txt"
                #print(source)
                
                
                destination_folder = dir_out+"/Topics_by_slices_2/slice_"+str(i)+"/"+str(topic)+"/"
                destination_slice_folder = dir_out+"/Topics_by_slices_2/slice_"+str(i)+"/"+"_all_files/"
                destination = dir_out+"/Topics_by_slices_2/slice_"+str(i)+"/"+str(topic)+"/"+filename+"_cleaned.txt"
                destination_slice = dir_out+"/Topics_by_slices_2/slice_"+str(i)+"/"+"_all_files/"+filename+"_cleaned.txt"
                
                
                if not os.path.exists(destination_folder):
                    os.makedirs(destination_folder)


                if not os.path.exists(destination_slice_folder):
                    os.makedirs(destination_slice_folder)
                    
                if not os.path.exists(destination_slice_folder):
                    os.makedirs(destination_slice)

                copyfile(source, destination)  # file is in a slice/topic folder to which it pertains
                copyfile(source, destination_slice) # file is in a "all files in this slice" folder. Removes duplicates

                total_number_files += 1
                number_files_slices[i] += 1


print("number of files moved",str(total_number_files))
print(number_files_slices)
#print(len(topic_slice_files[('6', 2)]))

number of files moved 3909
[2282, 1627]


In [16]:
print(dir_out)

/Users/hengchen/hartlib/OneDrive/files_years/la


In [18]:
# List of labels for the topics. Manually created by Robin
# this is hardcoded for now, next step is reading a file

### THIS IS FOR ENGLISH
#topic_labels = ["newsletters about politics", 
#               "friendship", 
#               "religion: spirituality/faith", 
#               "religion: church life", 
#               "religious war/peace", 
#               "Underspecified_actions",
#               "letter writing/copying", 
#               "trade of agricultural produce/plantations",
#               "advancement of learning for the public good",
#               "spirituality/dreams",
#               "making cider/horticulture",
#               "alchemy",
#               "visiting/travelling",
#               "poverty/charity(?)",
#               "(writing) books/manuscripts",
#               "personal relationships",
#               "medicine: medicinal herbs",
#               "education, learning languages",
#               "medicine: (iatro) chemical experiments",
#               "Office of Address (public office for jobs/knowledge)",
#               "army/military/troop movement",
#               "making an engine (water/manure)/technology/inventions",
#               "Underspecified_colonies/settlers",
#               "fear and hope",
#               "plans for innovations in fishing presented to Parliament",
#               "garbage topic",
#               "war/peace between nations",
#               "agriculture",
#               "religion: morality/sin",
#               "knowledge",
#               "letter writing/sending",
#               "law",
#               "religion: fear and hope",
#               "horticulture/husbandry",
#               "academic life/publishing of academic books",
#               "natural philosophy (physics)",
#               "money/selling and buying",
#               "religion: church/charity",
#               "religion: theology/doctrine",
#               "writing/editing (more about books than letters)"]

### THIS IS FOR LATIN
topic_labels_la = ["patronage for the public good","correspondence: exchanging news, books, etc.","pansophy/universal wisdom",
               "science/method","use of logic and rhetorics in theology","religion: Christian practice/sacraments",
               "inventions","logic/metaphysics","horticulture/plants","writing instruction","end time/millenarianism",
               "medicine: medical recipes","rhetoric: eulogies","correspondence: writing, sending and receiving letters",
               "peace negotiations/treaties","garbage topic/underspecified","logic: analysis and genesis",
               "didactics/theological instruction", "universal reformation","religion: sin, grace, free will",
               "communication","linguistics/teaching method","hope of better times","ecclesiastical peace: negotiations",
               "pansophy/didactics","iatrochemistry/medical alchemy","alchemy/metallurgy","apocalypse/prophesies",
                "pansophical college for gathering knowledge","rules of rhetoric","didactics/teaching method",
               "ecclesiastical peace: theological controversies","astrology/astronomy","logic: demonstration",
               "ecclesiastical peace: church unity","war","shipping/naval warfare",
               "religion: the essence of the Christian faith", "secrets","interstate war"]
#print(topic_labels)


### This below is for Latin
#for i in topic_labels:
#    la = "INCORRECT LABEL"+i
#    topic_labels_la.append(la)
    
#print(topic_abc)

## Visualising the number of files per topic per slice

In [19]:
##### CHANGE NUMBERS LINES 38 & 41


import pylab
import numpy as np
from array import array
import matplotlib.pyplot as plt


pylab.rcParams['figure.figsize'] = (10, 6) # this is for bigger figures

n_groups = len(slices)-1  # the number of bars we'll have, i.e the number of slices


for t in range(0,num_topics):
    
    bars = list()
    bar_width = 0.35
    bars_raw = list()

    
    
    
    for i in range(0,n_groups):

#        if (str(t),i) in topic_abc:
    
#            bars.append(1000*topic_abc[(str(t),i)]/number_files_slices[i])
            
            #number_files_slices[i] is the total number of files for slice i, with duplicates (one file in several topics)
#            bars_raw.append(topic_abc[(str(t),i)])


# above is the correct way with duplicates, here I am using unique numbers

        if (str(t),i) in topic_abc:
            if i == 0:
                bars.append(100*topic_abc[(str(t),i)]/1329) # 1329 = number of unique files in slice 0
            
            if i == 1:
                bars.append(100*topic_abc[(str(t),i)]/949) # 949 = number of unique files in slice 1

        else:
            bars.append(0) # In case we don't have anything, we store a zero
        bars_raw.append(topic_abc[(str(t),i)])

        
        
    print(t,bars)
    fig, ax = plt.subplots()

    index = np.arange(n_groups)

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = plt.bar(index, bars, bar_width,
                     alpha=opacity,
                     color='b',
                      #  yerr=std_men,
                     error_kw=error_config,
                     label='Files in topic in slice/Files in slice, in %')
    
    rects2 = plt.bar(index + bar_width, bars_raw, bar_width,
                 alpha=opacity,
                 color='r',
                 #yerr=std_women,
                 error_kw=error_config,
                 label='Raw count')
                     
    plt.xlabel('Slices, Total number of files per slice')
    plt.ylabel('Number of files assigned to this topic in each period')
    
    plt.title(topic_labels_la[t])
    #plt.xticks(index + bar_width / 2, (str(slices[i])+' to '+str(slices[i+1]), str(slices[1])+' to '+str(slices[2]), str(slices[2]) +' to' +str(slices[3])))
    plt.xticks(index + bar_width / 2)

    plt.legend()


    plt.savefig(dir_out+"/Topic "+str(t)+".png")
    #plt.savefig("ELEPHANT.PNG")

plt.show()
plt.gcf().clear()
print(topic_abc)
#plt.bar(list(topic_slice_number.keys()), dict_file_topic.values(), color='g')
#plt.title("Number of files per topic")
#plt.show()

#ax_f.set_ylabel('Fahrenheit')
#ax_c.set_ylabel('Celsius')

0 [7.524454477050414, 6.217070600632244]
1 [8.878856282919488, 6.954689146469969]
2 [3.762227238525207, 1.1591148577449948]
3 [3.310759969902182, 1.6859852476290833]
4 [12.415349887133182, 9.799789251844047]
5 [1.2039127163280663, 0.5268703898840885]
6 [2.18209179834462, 6.533192834562698]
7 [2.2573363431151243, 1.36986301369863]
8 [0.526711813393529, 2.6343519494204424]
9 [4.213694507148232, 2.845100105374078]
10 [0.9781790820165538, 2.107481559536354]
11 [0.8276899924755455, 4.004214963119073]
12 [6.922498118886381, 8.851422550052687]
13 [17.45673438675696, 20.969441517386723]
14 [3.8374717832957113, 5.268703898840885]
15 [5.568096313017306, 6.4278187565858795]
16 [5.568096313017306, 4.531085353003161]
17 [4.439428141459744, 3.6880927291886194]
18 [5.342362678705793, 4.7418335089567965]
19 [1.9563581640331076, 1.791359325605901]
20 [5.718585402558315, 6.8493150684931505]


/usr/local/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


21 [2.708803611738149, 0.5268703898840885]
22 [6.997742663656885, 9.483667017913593]
23 [8.051166290443943, 5.900948366701791]
24 [5.191873589164786, 3.161222339304531]
25 [0.526711813393529, 0.5268703898840885]
26 [0.8276899924755455, 9.062170706006322]
27 [0.8276899924755455, 2.4236037934668073]
28 [1.3544018058690745, 2.107481559536354]
29 [0.8276899924755455, 0.2107481559536354]
30 [2.3325808878856282, 1.6859852476290833]
31 [7.975921745673439, 2.3182297154899896]
32 [1.7306245297215952, 2.212855637513172]
33 [1.2039127163280663, 0.31612223393045313]
34 [9.255079006772009, 5.795574288724974]
35 [2.106847253574116, 6.006322444678609]
36 [1.2791572610985704, 1.36986301369863]
37 [6.696764484574868, 2.73972602739726]
38 [0.8276899924755455, 0.8429926238145417]
39 [6.094808126410835, 5.795574288724974]


<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

...

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

{('13', 0): 232, ('18', 0): 71, ('12', 0): 92, ('23', 0): 107, ('20', 0): 76, ('39', 0): 81, ('36', 0): 17, ('22', 0): 93, ('16', 0): 74, ('35', 0): 28, ('21', 0): 36, ('6', 0): 29, ('1', 0): 118, ('3', 0): 44, ('8', 0): 7, ('7', 0): 30, ('0', 0): 100, ('14', 0): 51, ('4', 0): 165, ('17', 0): 59, ('37', 0): 89, ('30', 0): 31, ('15', 0): 74, ('31', 0): 106, ('34', 0): 123, ('9', 0): 56, ('29', 0): 11, ('10', 0): 13, ('23', 1): 56, ('37', 1): 26, ('14', 1): 50, ('34', 1): 55, ('19', 1): 17, ('31', 1): 22, ('13', 1): 199, ('12', 1): 84, ('22', 1): 90, ('4', 1): 93, ('15', 1): 61, ('17', 1): 35, ('26', 0): 11, ('32', 0): 23, ('27', 0): 11, ('30', 1): 16, ('25', 0): 7, ('26', 1): 86, ('11', 1): 38, ('6', 1): 62, ('11', 0): 11, ('8', 1): 25, ('33', 1): 3, ('25', 1): 5, ('7', 1): 13, ('20', 1): 65, ('35', 1): 57, ('39', 1): 55, ('0', 1): 59, ('24', 0): 69, ('38', 1): 8, ('9', 1): 27, ('10', 1): 20, ('16', 1): 43, ('1', 1): 66, ('24', 1): 30, ('3', 1): 16, ('28', 1): 20, ('27', 1): 23, ('2', 1

## comment on "files in topic in slice / files in slice" :
### we do the division on the duplicates and not on the exact number of files (as one file is in several topics)

In [20]:
print(topic_abc[('0', 0)])

print(topic_year_number)

100
{('13', '1650'): 7, ('18', '1650'): 1, ('12', '1650'): 5, ('23', '1650'): 1, ('13', '1635'): 25, ('20', '1635'): 14, ('39', '1634'): 8, ('36', '1634'): 2, ('13', '1634'): 18, ('12', '1634'): 7, ('20', '1634'): 5, ('22', '1634'): 6, ('16', '1634'): 2, ('35', '1634'): 5, ('21', '1634'): 4, ('6', '1634'): 2, ('1', '1634'): 5, ('3', '1634'): 4, ('8', '1634'): 1, ('7', '1634'): 2, ('0', '1634'): 4, ('14', '1634'): 4, ('3', '1635'): 18, ('4', '1635'): 11, ('17', '1635'): 15, ('1', '1635'): 15, ('37', '1635'): 2, ('30', '1635'): 3, ('16', '1635'): 2, ('15', '1635'): 3, ('39', '1635'): 11, ('31', '1635'): 5, ('14', '1635'): 5, ('34', '1634'): 3, ('18', '1634'): 4, ('17', '1634'): 9, ('9', '1635'): 9, ('0', '1635'): 5, ('22', '1635'): 4, ('29', '1635'): 1, ('36', '1635'): 2, ('10', '1635'): 1, ('23', '1635'): 3, ('3', '1636'): 1, ('13', '1636'): 4, ('4', '1636'): 9, ('1', '1636'): 3, ('17', '1636'): 3, ('37', '1636'): 9, ('35', '1636'): 3, ('20', '1636'): 2, ('23', '1636'): 4, ('34', '1636'

In [22]:
threshold_of_files = 50 # this number is decided by us. Min number of files per topic in both slices

topic_threshold = list()

for t in range(0,num_topics):
   
    
    for i in range(0,n_groups):

        if (str(t),i) in topic_abc and (str(t),i+1) in topic_abc :
            if topic_abc[(str(t),i)] >= threshold_of_files and topic_abc[(str(t),i+1)] >= threshold_of_files:
                #print("topic, number of files for which both slices have more than threshold_of_files files:\n"+str(t),topic_abc[(str(t),i)],topic_abc[(str(t),i+1)])
                topic_threshold.append(t) 

print(topic_threshold)   
print(len(topic_threshold))

                
for topic in topic_threshold:
    print(topic,topic_labels_la[topic])


[0, 1, 4, 12, 13, 14, 15, 20, 22, 23, 34, 39]
12
0 patronage for the public good
1 correspondence: exchanging news, books, etc.
4 use of logic and rhetorics in theology
12 rhetoric: eulogies
13 correspondence: writing, sending and receiving letters
14 peace negotiations/treaties
15 garbage topic/underspecified
20 communication
22 hope of better times
23 ecclesiastical peace: negotiations
34 ecclesiastical peace: church unity
39 interstate war


## todo

- [x] two slices

- [x] filter correct years 
- [x] testing topic slice number
- [x] retrieve files pertaining to topics in different time slices 
- [x] visualisation -> will show what topics appeared and disappeared (account for number of files)

- [ ] run LDA with k = 1 on the slices for each of the 20 topics -- top words = 100, and we check the top 10 for changes

- [ ] for every topic find the difference between the words in slice 1 and the words in slice 2 for that topic
- [ ] consider trends in topic when interpreting results


- [x] normalise the number of letters per time slice (--> normalise number of files by all the files in that time slice) /_\ WE NEED CONFIDENCE INTERVAL
- [ ] optional: visualise the topic freq by time slice (+ find heuristic for "interesting trend", eg one slice is greater or equal than the sum of the two others)